## Assignment 1 Model Fitting with Maximum Likelihood 
* The purpose of this assignment is to learn how to write a log likelihood function, fit models to data with maximum likelihood and to select among models using AIC and a validation set.  
* Along the way, we'll learn a bit about decision making models and response time data.  

### Due: Sunday 10/19

In [1]:
import numpy as np
from numpy import random
from matplotlib import pyplot as plt
import pandas as pd
from scipy.optimize import minimize

### A Perceptual Discrimination Task to Study Decision Making

This is actual data from my lab, which was published here:

Nunez, M. D., Vandekerckhove, J., & Srinivasan, R. (2017). How attention influences perceptual decision making: Single-trial EEG correlates of drift-diffusion model parameters. Journal of Mathematical Psychology, 76(Part B), 117–130. https://doi.org/10.1016/j.jmp.2016.03.003

Human subject were asked to discriminate the spatial frequency of Gabor patches (as shown below), embeded in noise.  Task difficulty was controlled by the difficulty of the discrimination.  Two Gabors with more similar spatial frequencies are harder to discriminate, especially when noise is added.  In each of 34 participant, The experiment was performed in Easy, Medium, and Hard blocks each consisting of with decreasing differences between the Gabor spatial frequencies.  

![](spatialfrequency.png)

The datafile ReactionTimeData.csv is for use in this homework. You can load it into your notebook using pandas using pandas. There are 3 variables in the file: 

* Subject - indicates a numeric subject id 
* Experimental Condition - Easy, Medium, Hard 
* Correct - 1 if correct 0 if incorrect 
* ResponseTime  - time from stimulus presentation to decision in units of millisecond 

In [2]:
df = pd.read_csv('ResponseTimeData.csv')
rt = np.array(df['ResponseTime'])
correct = np.array(df['Correct'])
condition = np.array(df['Condition'])
condition_labels = ['Easy','Medium','Hard']

#### Problem 1 Explore and Visualize the Data.  I recommend using this exercise to learn about seaborn. 
a. make a histogram of Response Time showing all 3 difficulty conditions in a single graph.
 
 

b. make a bar graph showing the accuracy (proportion or percentage correct) in each condition.  

c. Make a boxplot that shows the distributions of each condition, with correct and incorrect trials separated. 

d. Calculate the mean reaction time for each subject in each condition.  Make a histogram that shows the distribution of mean RT across subjects,showing all 3 conditions in a single graph. 

In class we discussed how different distributions might be used to model Response Time data in decision making tasks.  In particular, the shifted Wald distribution is a distribution that captures aspects of the processes that give rise to Response Time.  

The shifted Wald distribution is a 1-boundary model. And, for simplicity, we will only consider correct trials.  



In [3]:
def shiftedwald(params, x):
    '''
    params: is a list or numpy array containing two parameters
    x: are the data
    '''
 
    gamma = params[0] #drift rate
    alpha = params[1] #boundary separation
    theta = params[2] #shift or nondecision time 
    x = x-theta
    z = alpha/np.sqrt(2*np.pi*(x**3))
    w = ((np.abs(alpha-gamma*x))**2)/(2*x)
    f = z*np.exp(-w)
    return f

### Problem 2 
Use the shifted Wald distribution defined above.  Write a function which computes the negative log likelihood of the shifted Wald distribution.  Assume that the data to be analyzed is in a variable called data. 

The function should return negative log likelihood 

In [4]:
def negloglikeWald(params):
    ''' written with params containing gamma, alpha, theta in that order.  Assumes data is in the variable data'''
    cprob = shiftedwald(params,data)
    cprob = np.maximum(cprob,1e-10) #to avoid log(0)
    negloglike = -np.sum(np.log(cprob))
    return negloglike 

### Problem 3 

Using the function the negative log-likelihood that you developed in problem 1, fit the shifted Wald distribution to the data for ALL the subjects in ReactionTimeData.csv.  

You should consider two models: 
Model A: All of the data comes from a single distribution. 
Model B: The data in each condition (Easy, Medium, Hard) comes from a separate distribution. 

When providing bounds to the fitting process, keep in mind that the only constraints on the parameters of this model is that they have to be positive.  To specify an upper bound of infinity, use `np.inf`

If you run into problems with your computer taking too long, reduce the amount of data, even do just 1 or a few subjects.

Make a plot or table or something to show me (make it pretty!) how the resulting parameter fits turned out.  

In [5]:
paramfit = dict()
data = rt[(correct == 1)]  #only use correct response 
bnds = ((0.01, np.inf), (0.01, np.inf), (0.01, np.min(data)-1))
paramfit['All'] = minimize(negloglikeWald, [0.1,100,200], bounds = bnds, options={'maxfun': 100000})
for c in condition_labels:
    data = rt[(correct == 1) & (condition == c)] 
    paramfit[c] = minimize(negloglikeWald, [0.1,100,200], bounds = bnds, options={'maxfun': 100000})

In [6]:
print(paramfit['All'])

  message: CONVERGENCE: RELATIVE REDUCTION OF F <= FACTR*EPSMCH
  success: True
   status: 0
      fun: 62486.859278231204
        x: [ 9.916e-02  6.493e+01  1.905e+02]
      nit: 21
      jac: [-1.150e-01 -9.459e-03  3.201e-02]
     nfev: 96
     njev: 24
 hess_inv: <3x3 LbfgsInvHessProduct with dtype=float64>


### Problem 4 

Use AIC to evaluate which model you should prefer.  Do the data come from 1 distribution or from three different distributions?  

In [7]:
aic = dict()
# for one model k = 3
k = 3
aic['A'] = 2*k + 2*paramfit['All']['fun']
# for 3 models k = 9 
k = 9
aic['B'] = 2*k + 2*(paramfit['Easy']['fun'] + paramfit['Medium']['fun'] + paramfit['Hard']['fun'])
print(aic)

{'A': np.float64(124979.71855646241), 'B': np.float64(124689.52488065843)}


### Problem 5 

The two models given above are not the only models we could propose. The power of likelihood and modeling (as opposed to statistical testing) is if we can be very specific about our hypothesis.    

The original idea of this experiment was to manipulate drift rate in order to find brain activity related to speed of information processing. 
So, our hypothesis was that non-decision time, and boundary would be the same for all conditions, and only the drift rate would vary.  

Write a function for negative log likelihood for a model that keeps boundary (alpha) and non-decision time (theta) the same for all 3 conditions, but allows drift rate (gamma) to vary between conditions. 

In this model there are now 5 parameters - gamma_easy, gamma_medium, gamma_hard, alpha, theta. 
The likelihood should be evaluated using the correct gamma for each condition, and using the sama alpha and theta for all condition.  

In [23]:
def negloglikeWald_C(params):
    ''' written with params containing gamma_easy, gamma_medium, gamma_hard, alpha, theta in that order.  Assumes data is in the variable data'''
    cprob = dict()
    negloglike = 0
    for i,c in enumerate(condition_labels):
        data_i = data[(accuracy == 1) & (clabel == c)] 
        params_i = [params[i], params[3], params[4]] # gamma for condition i, alpha, theta
        cprob = shiftedwald(params_i,data_i)
        cprob = np.maximum(cprob,1e-10) #to avoid log(0)
        negloglike = negloglike-np.sum(np.log(cprob))
    return negloglike 

### Problem 6

Use you new likelihood you developed in Problem 5 to fit the data (call it model C), and compare to models A and B using AIC. Make a new table showing the parameters for each model and which model seems to fit the data the best. (by AIC)   

In [9]:
data = rt  #use all response 
accuracy = correct
clabel = condition
bnds = ((0.01, np.inf), (0.01, np.inf), (0.01, np.inf), (0.01, np.inf), (0.01, np.min(data)-1))
paramfit['Optimal'] = minimize(negloglikeWald_C, [0.1,0.1,0.1,100,200], bounds = bnds, options={'maxfun': 100000})


In [10]:
print(paramfit['Optimal'])

  message: CONVERGENCE: RELATIVE REDUCTION OF F <= FACTR*EPSMCH
  success: True
   status: 0
      fun: 62339.34035566585
        x: [ 1.094e-01  1.002e-01  9.192e-02  6.595e+01  1.907e+02]
      nit: 23
      jac: [-1.019e-02  1.819e-02  2.183e-03 -3.711e-02  1.361e-01]
     nfev: 162
     njev: 27
 hess_inv: <5x5 LbfgsInvHessProduct with dtype=float64>


In [11]:
aic['C'] = 2*5 + 2*paramfit['Optimal']['fun']  #k = 5
print(aic)

{'A': np.float64(124979.71855646241), 'B': np.float64(124689.52488065843), 'C': np.float64(124688.6807113317)}


### Problem 7 (Experimental) - not required.  

Use sklearn's test_train_split to split the dataframe you read from ResponseTime.csv into training and test dataframes.  Keep around 0.2 or 0.3 of the data for testing.  When you do this, remember to stratify by participant!  Fit models A,B,C to the training data.  Then choose the best parameter values for each model to compute the likelihood of each model and identify which model has maximum likelihood in the test data. No AIC needed!   

In [12]:
df.keys()

Index(['Unnamed: 0', 'Subject', 'Condition', 'Correct', 'ResponseTime'], dtype='object')

In [ ]:
from sklearn.model_selection import train_test_split
# key here is that I used the stratify option to ensure that the train and test sets have similar distribution of subjects  
df_train, df_test = train_test_split(df, test_size=0.2, stratify = df['Subject'],random_state=42)
#grab the data from the training set
rt_train = np.array(df_train['ResponseTime'])
correct_train = np.array(df_train['Correct'])
condition_train = np.array(df_train['Condition'])
#grab the data from the test set
rt_test = np.array(df_test['ResponseTime'])
correct_test = np.array(df_test['Correct'])
condition_test = np.array(df_test['Condition'])
#make labels
condition_labels = ['Easy','Medium','Hard']

In [14]:
df_train['Subject'].value_counts()

Subject
22    284
32    284
16    284
9     284
8     284
10    283
4     282
3     282
27    282
17    282
28    282
23    282
15    281
6     281
31    280
13    280
11    278
18    277
24    276
19    276
7     275
20    275
26    274
25    274
5     272
30    272
34    270
29    270
12    269
2     266
33    263
14    254
21    246
1     163
Name: count, dtype: int64

In [15]:
df_test['Subject'].value_counts()

Subject
22    71
15    71
16    71
10    71
32    71
9     71
4     71
8     71
17    70
23    70
31    70
3     70
6     70
28    70
13    70
27    70
11    70
25    69
18    69
24    69
7     69
19    69
20    69
29    68
34    68
5     68
26    68
30    68
12    67
33    66
2     66
14    63
21    62
1     41
Name: count, dtype: int64

In [16]:
paramfit_train = dict()
data = rt_train[(correct_train == 1)]  #only use correct response 
bnds = ((0.01, np.inf), (0.01, np.inf), (0.01, np.min(data)-1))
paramfit_train['All'] = minimize(negloglikeWald, [0.1,100,200], bounds = bnds, options={'maxfun': 100000})
for c in condition_labels:
    data = rt_train[(correct_train == 1) & (condition_train == c)] 
    paramfit_train[c] = minimize(negloglikeWald, [0.1,100,200], bounds = bnds, options={'maxfun': 100000})


In [17]:
print("Fitted parameters for all data: gamma, alpha, theta")
print(paramfit['All']['x'])
print("Fitted parameters for training data: gamma, alpha, theta")
print(paramfit_train['All']['x'])

Fitted parameters for all data: gamma, alpha, theta
[9.91617866e-02 6.49312830e+01 1.90482827e+02]
Fitted parameters for training data: gamma, alpha, theta
[9.87862566e-02 6.48076208e+01 1.90494342e+02]


In [18]:
for c in condition_labels:
    print(f"Fitted parameters for {c} data: gamma, alpha, theta")
    print(paramfit[c]['x'])
    print(f"Fitted parameters for {c} training data: gamma, alpha, theta")
    print(paramfit_train[c]['x'])   

Fitted parameters for Easy data: gamma, alpha, theta
[1.07165862e-01 6.46756428e+01 1.89931822e+02]
Fitted parameters for Easy training data: gamma, alpha, theta
[1.07373273e-01 6.47552958e+01 1.89994944e+02]
Fitted parameters for Medium data: gamma, alpha, theta
[1.00306572e-01 6.60113518e+01 1.90609900e+02]
Fitted parameters for Medium training data: gamma, alpha, theta
[1.00033752e-01 6.60329877e+01 1.90661473e+02]
Fitted parameters for Hard data: gamma, alpha, theta
[9.44810595e-02 6.76911617e+01 1.91756188e+02]
Fitted parameters for Hard training data: gamma, alpha, theta
[9.35386547e-02 6.71912874e+01 1.91622777e+02]


In [19]:
data = rt_train  #use all response
accuracy = correct_train
clabel = condition_train
bnds = ((0.01, np.inf), (0.01, np.inf), (0.01, np.inf), (0.01, np.inf), (0.01, np.min(data)-1))
paramfit_train['Optimal'] = minimize(negloglikeWald_C, [0.1,0.1,0.1,100,200], bounds = bnds, options={'maxfun': 100000})


In [20]:
print("Fitted parameters for all data, Optimal model: gamma_easy, gamma_medium,gamma_hard, alpha, theta")
print(paramfit['Optimal']['x'])
print("Fitted parameters for training data, Optimal model: gamma_easy, gamma_medium,gamma_hard, alpha, theta")
print(paramfit_train['Optimal']['x'])

Fitted parameters for all data, Optimal model: gamma_easy, gamma_medium,gamma_hard, alpha, theta
[1.09417357e-01 1.00229615e-01 9.19174624e-02 6.59546394e+01
 1.90658494e+02]
Fitted parameters for training data, Optimal model: gamma_easy, gamma_medium,gamma_hard, alpha, theta
[1.09333783e-01 9.97831006e-02 9.15713422e-02 6.58642253e+01
 1.90670286e+02]


In [21]:
Likelihood_test = dict()
data = rt_test[(correct_test == 1)]  #only use correct response
Likelihood_test['A'] = negloglikeWald(paramfit_train['All']['x'])
data = rt_test
accuracy = correct_test
clabel = condition_test
Likelihood_test['C'] = negloglikeWald_C(paramfit_train['Optimal']['x'])
Likelihood_test['B'] = 0                                 
for c in condition_labels:
    data = rt_test[(correct_test == 1) & (condition_test == c)]
    Likelihood_test['B'] = Likelihood_test['B'] + negloglikeWald(paramfit_train[c]['x'])


In [22]:
print(Likelihood_test)

{'A': np.float64(12446.58906683985), 'C': np.float64(12421.126925877485), 'B': np.float64(12419.595278913697)}
